In [744]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import timedelta

## Database Connection

In [745]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [746]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [747]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [748]:
dim_fase = pd.read_sql_table('dim_fase_servicio', etl_conn) 
fecha = pd.read_sql_table('dim_fecha', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
estado_servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

## organizar datos

In [749]:
servicio = servicio[["id", "fecha_solicitud", "hora_solicitud", "mensajero_id"]]
servicio.head(5)

,id,fecha_solicitud,hora_solicitud,mensajero_id
0,34,2023-10-26,09:46:03,NaN
1,35,2023-10-26,11:18:14,7.0
2,36,2023-10-28,19:21:01,NaN
3,41,2023-11-07,09:46:09,NaN
4,42,2023-11-07,09:46:10,NaN


In [752]:
estado_servicio.drop(columns=["foto","observaciones","es_prueba","foto_binary"], inplace=True)


In [753]:
estado_servicio[estado_servicio['estado_id'].isin([1, 2, 3, 4, 5, 6])]['estado_id'].value_counts()
estado_servicio_final = estado_servicio
estado_servicio.head(5)

,id,fecha,hora,estado_id,servicio_id
0,1014,2024-01-29,01:13:32,4,226
1,1484,2024-01-30,18:45:12,3,79
2,2829,2024-02-06,11:34:04,5,613
3,1888,2024-02-01,14:50:39,4,376
4,32312,2024-04-06,16:11:21,3,7164


In [754]:
total_nan = estado_servicio_final.isna().sum()

# Asegurarnos de que las columnas de fecha y hora son de tipo string
estado_servicio_final['fecha'] = estado_servicio_final['fecha'].astype(str)
estado_servicio_final['hora'] = estado_servicio_final['hora'].astype(str)

# Truncar los milisegundos (decimales) de la columna 'hora' si existen
estado_servicio_final['hora'] = estado_servicio_final['hora'].str.split('.').str[0]

# Convertir las columnas de fecha y hora en una única columna de tipo datetime
estado_servicio_final['datetime'] = pd.to_datetime(estado_servicio_final['fecha'] + ' ' + estado_servicio_final['hora'], errors='coerce')

# Ordenar el DataFrame por 'servicio_id', 'estado_id' y 'datetime' de forma descendente
estado_servicio_final = estado_servicio_final.sort_values(by=['servicio_id', 'estado_id', 'datetime'], ascending=[True, True, False])

# Mostrar el resultado
estado_servicio_final.head()



,id,fecha,hora,estado_id,servicio_id,datetime
301,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18
667,117,2023-10-13,17:51:20,2,7,2023-10-13 17:51:20
686,134,2023-10-31,12:02:48,4,7,2023-10-31 12:02:48
687,135,2023-10-31,17:07:55,5,7,2023-10-31 17:07:55
688,136,2023-10-31,12:16:00,6,7,2023-10-31 12:16:00


In [755]:
print(len(estado_servicio_final))

128402


## Transformation

### Date Process

In [756]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
estado_servicio_final['fecha'] = pd.to_datetime(estado_servicio_final['fecha'], errors='coerce')

estado_servicio_final = pd.merge(
    estado_servicio_final,
    fecha[["date", "key_fecha"]],
    left_on="fecha",
    right_on="date",
    how="left"
)

print(servicio.columns)
print(mensajero.columns)

servicio = pd.merge(servicio, mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
servicio.head(5)

Index(['id', 'fecha_solicitud', 'hora_solicitud', 'mensajero_id'], dtype='object')
Index(['key_mensajero', 'mensajero_id', 'activo', 'salario',
       'ciudad_operacion', 'departamento_operacion', 'first_name', 'last_name',
       'email', 'saved'],
      dtype='object')


,id,fecha_solicitud,hora_solicitud,mensajero_id,key_mensajero
0,34,2023-10-26,09:46:03,NaN,NaN
1,35,2023-10-26,11:18:14,7.0,13.0
2,36,2023-10-28,19:21:01,NaN,NaN
3,41,2023-11-07,09:46:09,NaN,NaN
4,42,2023-11-07,09:46:10,NaN,NaN


### Phase Process

In [757]:
fase_iniciado = estado_servicio_final[estado_servicio_final['estado_id'] == 1]
fase_mensajero_asignado = estado_servicio_final[estado_servicio_final['estado_id'] == 2]
fase_recogido_mensajero = estado_servicio_final[estado_servicio_final['estado_id'] == 4]
fase_entregado_destino = estado_servicio_final[estado_servicio_final['estado_id'] == 5]
fase_terminado = estado_servicio_final[estado_servicio_final['estado_id'] == 6]


In [758]:

# Ordenar el DataFrame por 'servicio_id', 'estado_id' y 'datetime' de forma descendente
fase_iniciado = fase_iniciado.sort_values(by=['servicio_id', 'estado_id', 'datetime'], ascending=[True, True, True])


In [759]:

fase_iniciado.rename(columns={'fecha':'fecha_inicio', 'hora':'hora_inicio', 'key_fecha':'key_fecha_inicio'}, inplace=True)
fase_iniciado = fase_iniciado[['servicio_id','fecha_inicio','hora_inicio','key_fecha_inicio','estado_id']]
fase_iniciado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado'}, inplace=True)
fase_mensajero_asignado = fase_mensajero_asignado[['servicio_id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado','estado_id']]
fase_mensajero_asignado.rename(columns={'servicio_id':'id'}, inplace=True)

fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajero'}, inplace=True)
fase_recogido_mensajero = fase_recogido_mensajero[['servicio_id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero','estado_id']]
fase_recogido_mensajero.rename(columns={'servicio_id':'id'}, inplace=True)

fase_entregado_destino.rename(columns={'fecha':'fecha_entregado_destino', 'hora':'hora_entregado_destino','key_fecha':'key_fecha_entregado_destino'}, inplace=True)
fase_entregado_destino = fase_entregado_destino[['servicio_id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino','estado_id']]
fase_entregado_destino.rename(columns={'servicio_id':'id'}, inplace=True)

fase_terminado.rename(columns={'fecha':'fecha_terminado', 'hora':'hora_terminado','key_fecha':'key_fecha_terminado'}, inplace=True)
fase_terminado = fase_terminado[['servicio_id','fecha_terminado','hora_terminado','key_fecha_terminado','estado_id']]
fase_terminado.rename(columns={'servicio_id':'id'}, inplace=True)

C:\Users\deica\AppData\Local\Temp\ipykernel_5024\1457959828.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_mensajero_asignado.rename(columns={'fecha':'fecha_mensajero_asignado', 'hora':'hora_mensajero_asignado', 'key_fecha':'key_fecha_mensajero_asignado'}, inplace=True)
C:\Users\deica\AppData\Local\Temp\ipykernel_5024\1457959828.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fase_recogido_mensajero.rename(columns={'fecha':'fecha_recogido_mensajero', 'hora':'hora_recogido_mensajero','key_fecha':'key_fecha_recogido_mensajero'}, inplace=True)
C:\Users\deica\AppData\Local\Temp\ipykernel_5024\1457959828.py:13

## Merge de datos

In [760]:

servicio = pd.merge(estado_servicio_final, fase_iniciado[['id', 'fecha_inicio', 'hora_inicio', 'key_fecha_inicio']], left_on="servicio_id", right_on="id", how='left',suffixes=("","id"))
servicio = servicio.drop(columns=['idid'], errors='ignore')

servicio = pd.merge(servicio, fase_mensajero_asignado[['id','fecha_mensajero_asignado','hora_mensajero_asignado','key_fecha_mensajero_asignado']], left_on="servicio_id", right_on="id", how="left",suffixes=("","id"))
servicio = servicio.drop(columns=['idid'], errors='ignore')

servicio = pd.merge(servicio, fase_recogido_mensajero[['id','fecha_recogido_mensajero','hora_recogido_mensajero','key_fecha_recogido_mensajero']], left_on="servicio_id", right_on="id", how="left",suffixes=("","id"))
servicio = servicio.drop(columns=['idid'], errors='ignore')

servicio = pd.merge(servicio, fase_entregado_destino[['id','fecha_entregado_destino','hora_entregado_destino','key_fecha_entregado_destino']], left_on="servicio_id", right_on="id", how="left",suffixes=("","id"))
servicio = servicio.drop(columns=['idid'], errors='ignore')

servicio = pd.merge(servicio, fase_terminado[['id','fecha_terminado','hora_terminado','key_fecha_terminado']], left_on="servicio_id", right_on="id", how="left",suffixes=("","id"))
servicio = servicio.drop(columns=['idid'], errors='ignore')


In [761]:
servicio.head(5)

,id,fecha,hora,estado_id,servicio_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado
0,6,2023-09-19,16:22:18,1,7,2023-09-19 16:22:18,2023-09-19,253,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0
1,117,2023-10-13,17:51:20,2,7,2023-10-13 17:51:20,2023-10-13,277,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0
2,134,2023-10-31,12:02:48,4,7,2023-10-31 12:02:48,2023-10-31,295,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0
3,135,2023-10-31,17:07:55,5,7,2023-10-31 17:07:55,2023-10-31,295,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0
4,136,2023-10-31,12:16:00,6,7,2023-10-31 12:16:00,2023-10-31,295,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0


## Tomar un registro por servicio

In [762]:
servicio_unique = servicio.groupby(['servicio_id'], as_index=False).first()

servicio_unique.head(5)

,servicio_id,id,fecha,hora,estado_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado
0,7,6,2023-09-19,16:22:18,1,2023-09-19 16:22:18,2023-09-19,253,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0
1,8,7,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,16:30:05,253.0,2023-12-20,20:14:43,345.0,2024-02-14,15:34:18,401.0,2024-04-09,16:08:35,456.0,NaT,None,NaN
2,9,8,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,16:30:05,253.0,2023-12-28,19:33:01,353.0,NaT,None,NaN,NaT,None,NaN,NaT,None,NaN
3,10,9,2023-09-19,16:35:52,1,2023-09-19 16:35:52,2023-09-19,253,2023-09-19,16:35:52,253.0,2023-12-28,19:33:07,353.0,2024-02-18,00:21:47,405.0,2024-03-10,09:58:27,426.0,NaT,None,NaN
4,11,10,2023-09-19,16:37:54,1,2023-09-19 16:37:54,2023-09-19,253,2023-09-19,16:37:54,253.0,2023-12-09,13:13:59,334.0,2024-01-31,10:29:55,387.0,NaT,None,NaN,NaT,None,NaN


In [763]:
# Ajustar el número de filas y columnas a mostrar
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

servicio_id_7 = servicio_unique.loc[servicio_unique['servicio_id'] == 26]
servicio_id_7.head(1)


,servicio_id,id,fecha,hora,estado_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado
19,26,63,2023-10-05,22:33:07,1,2023-10-05 22:33:07,2023-10-05,269,2023-09-23,19:25:56,257.0,2023-10-07,16:30:09,271.0,2023-10-07,16:30:11,271.0,2023-10-07,16:30:21,271.0,NaT,None,NaN


In [764]:

len(servicio_unique)

28430

### Calculo días

In [765]:
servicio_unique['dias_mensajero_asignado'] = (servicio_unique['fecha_mensajero_asignado'] - servicio_unique['fecha_inicio']).dt.days.fillna(0)
servicio_unique['dias_recogido_mensajero'] = (servicio_unique['fecha_recogido_mensajero'] - servicio_unique['fecha_mensajero_asignado']).dt.days.fillna(0)
servicio_unique['dias_entregado_destino'] = (servicio_unique['fecha_entregado_destino'] - servicio_unique['fecha_recogido_mensajero']).dt.days.fillna(0)
servicio_unique['dias_terminado'] = (servicio_unique['fecha_terminado'] - servicio_unique['fecha_entregado_destino']).dt.days.fillna(0)
# Calcular el promedio general de entrega en días y redondear a 2 decimales
servicio_unique['promedio_entrega_general'] = (
    (servicio_unique['dias_mensajero_asignado'] + 
     servicio_unique['dias_recogido_mensajero'] + 
     servicio_unique['dias_entregado_destino'] + 
     servicio_unique['dias_terminado']) / 4
).round(2)
servicio_unique.head(5)

,servicio_id,id,fecha,hora,estado_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general
0,7,6,2023-09-19,16:22:18,1,2023-09-19 16:22:18,2023-09-19,253,2023-09-19,16:22:18,253.0,2023-10-13,17:51:20,277.0,2023-10-31,12:02:48,295.0,2023-10-31,17:07:55,295.0,2023-10-31,12:16:00,295.0,24.0,18.0,0.0,0.0,10.50
1,8,7,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,16:30:05,253.0,2023-12-20,20:14:43,345.0,2024-02-14,15:34:18,401.0,2024-04-09,16:08:35,456.0,NaT,None,NaN,92.0,56.0,55.0,0.0,50.75
2,9,8,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,16:30:05,253.0,2023-12-28,19:33:01,353.0,NaT,None,NaN,NaT,None,NaN,NaT,None,NaN,100.0,0.0,0.0,0.0,25.00
3,10,9,2023-09-19,16:35:52,1,2023-09-19 16:35:52,2023-09-19,253,2023-09-19,16:35:52,253.0,2023-12-28,19:33:07,353.0,2024-02-18,00:21:47,405.0,2024-03-10,09:58:27,426.0,NaT,None,NaN,100.0,52.0,21.0,0.0,43.25
4,11,10,2023-09-19,16:37:54,1,2023-09-19 16:37:54,2023-09-19,253,2023-09-19,16:37:54,253.0,2023-12-09,13:13:59,334.0,2024-01-31,10:29:55,387.0,NaT,None,NaN,NaT,None,NaN,81.0,53.0,0.0,0.0,33.50


### Cálculo Hora

In [766]:
# Convertir las columnas a tipo datetime con solo el componente de hora
servicio_unique['hora_inicio'] = pd.to_datetime(servicio_unique['hora_inicio'], format='%H:%M:%S', errors='coerce')
servicio_unique['hora_mensajero_asignado'] = pd.to_datetime(servicio_unique['hora_mensajero_asignado'], format='%H:%M:%S', errors='coerce')
servicio_unique['hora_recogido_mensajero'] = pd.to_datetime(servicio_unique['hora_recogido_mensajero'], format='%H:%M:%S', errors='coerce')
servicio_unique['hora_entregado_destino'] = pd.to_datetime(servicio_unique['hora_entregado_destino'], format='%H:%M:%S', errors='coerce')
servicio_unique['hora_terminado'] = pd.to_datetime(servicio_unique['hora_terminado'], format='%H:%M:%S', errors='coerce')

# Calcular las diferencias en horas, redondear, llenar nulos con 0, y tomar valor absoluto
servicio_unique['horas_mensajero_asignado'] = ((servicio_unique['hora_mensajero_asignado'] - servicio_unique['hora_inicio']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio_unique['horas_recogido_mensajero'] = ((servicio_unique['hora_recogido_mensajero'] - servicio_unique['hora_mensajero_asignado']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio_unique['horas_entregado_destino'] = ((servicio_unique['hora_entregado_destino'] - servicio_unique['hora_recogido_mensajero']).dt.total_seconds() / 3600).round(2).fillna(0).abs()
servicio_unique['horas_terminado'] = ((servicio_unique['hora_terminado'] - servicio_unique['hora_entregado_destino']).dt.total_seconds() / 3600).round(2).fillna(0).abs()

servicio_unique.head(5)


,servicio_id,id,fecha,hora,estado_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado
0,7,6,2023-09-19,16:22:18,1,2023-09-19 16:22:18,2023-09-19,253,2023-09-19,1900-01-01 16:22:18,253.0,2023-10-13,1900-01-01 17:51:20,277.0,2023-10-31,1900-01-01 12:02:48,295.0,2023-10-31,1900-01-01 17:07:55,295.0,2023-10-31,1900-01-01 12:16:00,295.0,24.0,18.0,0.0,0.0,10.50,1.48,5.81,5.09,4.87
1,8,7,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,1900-01-01 16:30:05,253.0,2023-12-20,1900-01-01 20:14:43,345.0,2024-02-14,1900-01-01 15:34:18,401.0,2024-04-09,1900-01-01 16:08:35,456.0,NaT,NaT,NaN,92.0,56.0,55.0,0.0,50.75,3.74,4.67,0.57,0.00
2,9,8,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19,1900-01-01 16:30:05,253.0,2023-12-28,1900-01-01 19:33:01,353.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,100.0,0.0,0.0,0.0,25.00,3.05,0.00,0.00,0.00
3,10,9,2023-09-19,16:35:52,1,2023-09-19 16:35:52,2023-09-19,253,2023-09-19,1900-01-01 16:35:52,253.0,2023-12-28,1900-01-01 19:33:07,353.0,2024-02-18,1900-01-01 00:21:47,405.0,2024-03-10,1900-01-01 09:58:27,426.0,NaT,NaT,NaN,100.0,52.0,21.0,0.0,43.25,2.95,19.19,9.61,0.00
4,11,10,2023-09-19,16:37:54,1,2023-09-19 16:37:54,2023-09-19,253,2023-09-19,1900-01-01 16:37:54,253.0,2023-12-09,1900-01-01 13:13:59,334.0,2024-01-31,1900-01-01 10:29:55,387.0,NaT,NaT,NaN,NaT,NaT,NaN,81.0,53.0,0.0,0.0,33.50,3.40,2.73,0.00,0.00


## Rellenar datos faltantes

In [774]:
servicio_unique.fillna(-1, inplace=True)
servicio_unique.head(10)

,servicio_id,id,fecha,hora,estado_id,datetime,date,key_fecha,fecha_inicio,hora_inicio,key_fecha_inicio,fecha_mensajero_asignado,hora_mensajero_asignado,key_fecha_mensajero_asignado,fecha_recogido_mensajero,hora_recogido_mensajero,key_fecha_recogido_mensajero,fecha_entregado_destino,hora_entregado_destino,key_fecha_entregado_destino,fecha_terminado,hora_terminado,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado
0,7,6,2023-09-19,16:22:18,1,2023-09-19 16:22:18,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:22:18,253.0,2023-10-13 00:00:00,1900-01-01 17:51:20,277.0,2023-10-31 00:00:00,1900-01-01 12:02:48,295.0,2023-10-31 00:00:00,1900-01-01 17:07:55,295.0,2023-10-31 00:00:00,1900-01-01 12:16:00,295.0,24.0,18.0,0.0,0.0,10.50,1.48,5.81,5.09,4.87
1,8,7,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:30:05,253.0,2023-12-20 00:00:00,1900-01-01 20:14:43,345.0,2024-02-14 00:00:00,1900-01-01 15:34:18,401.0,2024-04-09 00:00:00,1900-01-01 16:08:35,456.0,-1,-1,-1.0,92.0,56.0,55.0,0.0,50.75,3.74,4.67,0.57,0.00
2,9,8,2023-09-19,16:30:05,1,2023-09-19 16:30:05,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:30:05,253.0,2023-12-28 00:00:00,1900-01-01 19:33:01,353.0,-1,-1,-1.0,-1,-1,-1.0,-1,-1,-1.0,100.0,0.0,0.0,0.0,25.00,3.05,0.00,0.00,0.00
3,10,9,2023-09-19,16:35:52,1,2023-09-19 16:35:52,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:35:52,253.0,2023-12-28 00:00:00,1900-01-01 19:33:07,353.0,2024-02-18 00:00:00,1900-01-01 00:21:47,405.0,2024-03-10 00:00:00,1900-01-01 09:58:27,426.0,-1,-1,-1.0,100.0,52.0,21.0,0.0,43.25,2.95,19.19,9.61,0.00
4,11,10,2023-09-19,16:37:54,1,2023-09-19 16:37:54,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:37:54,253.0,2023-12-09 00:00:00,1900-01-01 13:13:59,334.0,2024-01-31 00:00:00,1900-01-01 10:29:55,387.0,-1,-1,-1.0,-1,-1,-1.0,81.0,53.0,0.0,0.0,33.50,3.40,2.73,0.00,0.00
5,12,11,2023-09-19,16:49:17,1,2023-09-19 16:49:17,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:49:17,253.0,2023-10-07 00:00:00,1900-01-01 13:47:47,271.0,2023-10-29 00:00:00,1900-01-01 00:41:36,293.0,2023-10-29 00:00:00,1900-01-01 00:44:22,293.0,2023-10-29 00:00:00,1900-01-01 00:46:00,293.0,18.0,22.0,0.0,0.0,10.00,3.02,13.10,0.05,0.03
6,13,12,2023-09-19,16:52:14,1,2023-09-19 16:52:14,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:52:14,253.0,2023-10-07 00:00:00,1900-01-01 13:08:30,271.0,2023-10-13 00:00:00,1900-01-01 23:55:07,277.0,2024-01-02 00:00:00,1900-01-01 03:32:02,358.0,2024-01-02 00:00:00,1900-01-01 03:32:31,358.0,18.0,6.0,81.0,0.0,26.25,3.73,10.78,20.38,0.01
7,14,13,2023-09-19,16:59:50,1,2023-09-19 16:59:50,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 16:59:50,253.0,2023-10-07 00:00:00,1900-01-01 13:07:23,271.0,2024-02-22 00:00:00,1900-01-01 16:00:36,409.0,2024-02-23 00:00:00,1900-01-01 12:09:52,410.0,2024-03-26 00:00:00,1900-01-01 16:22:36,442.0,18.0,138.0,1.0,32.0,47.25,3.87,2.89,3.85,4.21
8,15,14,2023-09-19,17:03:49,1,2023-09-19 17:03:49,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 17:03:49,253.0,2023-10-07 00:00:00,1900-01-01 12:53:33,271.0,-1,-1,-1.0,-1,-1,-1.0,-1,-1,-1.0,18.0,0.0,0.0,0.0,4.50,4.17,0.00,0.00,0.00
9,16,15,2023-09-19,17:07:28,1,2023-09-19 17:07:28,2023-09-19,253,2023-09-19 00:00:00,1900-01-01 17:07:28,253.0,2023-10-07 00:00:00,1900-01-01 12:52:03,271.0,2024-01-02 00:00:00,1900-01-01 03:36:10,358.0,2024-01-02 00:00:00,1900-01-01 03:36:37,358.0,2024-01-02 00:00:00,1900-01-01 03:36:56,358.0,18.0,87.0,0.0,0.0,26.25,4.26,9.26,0.01,0.01


## Eliminar columnas no necesarias

In [783]:
servicio_unique = servicio_unique.drop(columns=['fecha',"hora","estado_id","datetime","date", 
                                                "fecha_inicio","hora_inicio","servicio_id",
                                                "fecha_mensajero_asignado","hora_mensajero_asignado",
                                                "fecha_recogido_mensajero", "hora_recogido_mensajero",
                                                "fecha_entregado_destino","hora_entregado_destino",
                                                "fecha_terminado", "hora_terminado","id"], errors='ignore')
servicio_unique.head(10)

,key_fecha,key_fecha_inicio,key_fecha_mensajero_asignado,key_fecha_recogido_mensajero,key_fecha_entregado_destino,key_fecha_terminado,dias_mensajero_asignado,dias_recogido_mensajero,dias_entregado_destino,dias_terminado,promedio_entrega_general,horas_mensajero_asignado,horas_recogido_mensajero,horas_entregado_destino,horas_terminado
0,253,253.0,277.0,295.0,295.0,295.0,24.0,18.0,0.0,0.0,10.50,1.48,5.81,5.09,4.87
1,253,253.0,345.0,401.0,456.0,-1.0,92.0,56.0,55.0,0.0,50.75,3.74,4.67,0.57,0.00
2,253,253.0,353.0,-1.0,-1.0,-1.0,100.0,0.0,0.0,0.0,25.00,3.05,0.00,0.00,0.00
3,253,253.0,353.0,405.0,426.0,-1.0,100.0,52.0,21.0,0.0,43.25,2.95,19.19,9.61,0.00
4,253,253.0,334.0,387.0,-1.0,-1.0,81.0,53.0,0.0,0.0,33.50,3.40,2.73,0.00,0.00
5,253,253.0,271.0,293.0,293.0,293.0,18.0,22.0,0.0,0.0,10.00,3.02,13.10,0.05,0.03
6,253,253.0,271.0,277.0,358.0,358.0,18.0,6.0,81.0,0.0,26.25,3.73,10.78,20.38,0.01
7,253,253.0,271.0,409.0,410.0,442.0,18.0,138.0,1.0,32.0,47.25,3.87,2.89,3.85,4.21
8,253,253.0,271.0,-1.0,-1.0,-1.0,18.0,0.0,0.0,0.0,4.50,4.17,0.00,0.00,0.00
9,253,253.0,271.0,358.0,358.0,358.0,18.0,87.0,0.0,0.0,26.25,4.26,9.26,0.01,0.01


In [781]:
len(servicio_unique)

28430

## Load

In [784]:
servicio_unique.to_sql("hecho_accumulating_snapshot", etl_conn, if_exists="replace", index_label="key_servicio_fase") 

430